In [17]:
### Imports
from nltk.cluster.util import cosine_distance
from nltk.corpus import stopwords
from nltk import sent_tokenize, word_tokenize
import numpy as np
import networkx as nx
from helpers import *
stop_words = set(stopwords.words('english'))

# Read in and Preprocess Data

In [3]:
## Read in data
test_file = open('test.txt','r')
test_text = test_file.read()
test_file.close()

In [4]:
## Preprocess
formatted_article_text, article_text = fix_it_up(test_text)

Formatted - no punctuation, will be used to parse words
Article Text - with punctuation, made to create sentence list to pull from at end

# Tokenization 

In [11]:
## Word IDF

word_freq, sent_list = sentence_tokenize(formatted_article_text, article_text, stop_words)
word_idf = word_idf_create(sent_list, word_freq)

In [ ]:
## tokenized sentence list

sent_broken = [sent_tokenize(sent) for sent in sent_list]
sent_broken

In [ ]:
## tokenized sentence list with words 

sent_cleaned = [word_tokenize(str(sent)) for sent in sent_broken]

# Create Sentence IDF Vectors, Cosine Similarity Matrix

In [26]:
test1 = sent_cleaned[5]
test2 = sent_cleaned[10]

In [32]:
def cosine_similarity(sent1, sent2, word_idf):
        
    all_words = list(set(sent1 + sent2))
    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)
    
    for word in sent1:
        if word in word_idf.keys():
            vector1[all_words.index(word)] += word_idf[word]
    
    for word in sent2:
        if word in word_idf.keys():
            vector2[all_words.index(word)] += word_idf[word]
    
    return 1 - cosine_distance(vector1, vector2)

In [33]:
test_result = cosine_similarity(test1, test2, word_idf)

Building Similarity Matrix - Doing this step by step:

In [36]:
similarity_matrix= np.zeros((len(sent_cleaned), len(sent_cleaned)))

In [38]:
for idx1 in range(len(sent_cleaned)):
    for idx2 in range(len(sent_cleaned)):
        if idx1 == idx2:
            continue
        similarity_matrix[idx1][idx2] = cosine_similarity(sent_cleaned[idx1], sent_cleaned[idx2], word_idf)

In [40]:
similarity_graph = nx.from_numpy_array(similarity_matrix)

In [41]:
scores = nx.pagerank(similarity_graph)

In [46]:
type(scores)

dict

In [42]:
ranked_sentence = sorted(((scores[i],s) for i,s in enumerate(sent_broken)), reverse=True)    

In [43]:
summarize_text = []

for i in range(15):
      summarize_text.append(" ".join(ranked_sentence[i][1]))

In [44]:
summarize_text

['During breeding season, the pandas would return to lower altitudes to eat bamboo leaves rich in calcium.',
 'Only a few bamboo species are widespread at the high altitudes pandas now inhabit.',
 'The mutation time for the T1R1 gene in the giant panda is estimated to 4.2 mya while fossil evidence indicates bamboo consumption in the giant panda species at least 7 mya, signifying that although complete herbivory occurred around 2 mya, the dietary switch was initiated prior to T1R1 loss-of-function.',
 'During the summer months, bamboo shoots rich in protein are only available at higher altitudes which causes low calcium rates in the pandas.',
 'because of the rapid passage of large amounts of indigestible plant material through the short, straight digestive tract.',
 'It has been estimated that an adult panda absorbs 54.8â€“66.1â€‰mg of cyanide a day through its diet.',
 'Because of the synchronous flowering, death, and regeneration of all bamboo within a species, the giant panda must h